In [ ]:
!pip install -q transformers accelerate safetensors pillow pandas

In [ ]:
from google.colab import drive, userdata
import os, re, time, csv
from pathlib import Path
import pandas as pd
from PIL import Image
from tqdm import tqdm
from google import genai


drive.mount("/content/drive")

api_key = userdata.get("GEMINI_API_KEY_TAVE")
print("API KEY loaded?", bool(api_key))
if not api_key:
    raise ValueError("GEMINI_API_KEY가 Colab Secrets에 없습니다. Secrets에 GEMINI_API_KEY로 추가하세요.")

client = genai.Client(api_key=api_key)


folder = Path("/content/drive/MyDrive/TAVE 16th 심화플젝/data/total_thumbnail_face_labeling_0.2")

def sort_key(p: Path):
    nums = re.findall(r"\d+", p.stem)
    return (int(nums[0]) if nums else 10**18, p.name.lower())

image_paths = sorted(
    [p for p in folder.iterdir() if p.suffix.lower() in [".jpg", ".jpeg", ".png"]],
    key=sort_key
)

print("총 이미지 개수:", len(image_paths))


# 프롬프트
prompt_base = """
이 이미지를 한국어로 자연스럽게 묘사해줘.

규칙:
1) 이미지 안에는 인물 이름이 텍스트로 표시되어 있다. 이 이름은 '배역명(배우명)' 형식일 수 있다.
2) 이름이 '배역명(배우명)' 형식일 경우, 괄호 앞의 **배역명만** 사용해서 행동을 묘사해줘.
   예: "기정(박소담)" → "기정" / "연교(조여정)" → "연교"
3) 이름이 "unknown" 또는 읽기 어려운 경우 → 이름을 언급하지 말고 '한 남성', '한 여성', '한 사람' 등으로 묘사해줘.
4) 바운딩박스, 숫자(정확도), 색상, 좌표 등 라벨 표시 방식에 대한 언급 금지.
5) 인물이 하는 행동, 장면 분위기, 현재 장소 중심으로 **보이는 사실만** 객관적으로 자세하게 묘사해줘.
6) 감정, 의도, 추론적 판단 금지. 보이는 것만 설명.
7) 출력은 **최대 4문장**으로 작성해줘.
"""


def generate_caption(img: Image.Image, max_retries=6):
    delay = 2.0
    last_msg = None

    for attempt in range(max_retries):
        try:
            resp = client.models.generate_content(
                model="gemini-2.5-flash",
                contents=[prompt_base, img],
                # config={"max_output_tokens": 128},  # 출력 토큰 제한
            )
            return resp.text, None

        except Exception as e:
            last_msg = str(e)

            # 레이트리밋/일시 장애면 백오프 재시도
            if ("429" in last_msg) or ("RESOURCE_EXHAUSTED" in last_msg) or ("503" in last_msg) or ("UNAVAILABLE" in last_msg):
                time.sleep(delay)
                delay = min(delay * 2, 60)
                continue

            return None, last_msg

    return None, f"retry_exceeded (last_delay={delay}) | last_error={last_msg}"


START_IDX = 12
END_IDX = 988
CHUNK_SIZE = 100

END_IDX = min(END_IDX, len(image_paths))
if START_IDX > END_IDX:
    raise ValueError(f"START_IDX({START_IDX})가 END_IDX({END_IDX})보다 큽니다. 이미지 개수를 확인하세요.")

SLEEP_PER_CALL = 13


def load_done_filenames(csv_path: Path):
    if not csv_path.exists():
        return set()
    try:
        df = pd.read_csv(csv_path)
        if not {"filename", "caption", "error"}.issubset(df.columns):
            return set()

        ok = df[
            df["caption"].notna() &
            (df["caption"].astype(str).str.strip() != "") &
            (df["error"].isna() | (df["error"].astype(str).str.strip() == ""))
        ]
        return set(ok["filename"].astype(str).tolist())
    except Exception:
        return set()

def append_row(csv_path: Path, row: dict):
    file_exists = csv_path.exists()
    with open(csv_path, "a", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=["index", "filename", "caption", "error"])
        if not file_exists:
            writer.writeheader()
        writer.writerow(row)


# 실행
target_paths = image_paths[START_IDX-1:END_IDX]
print(f"총 {len(image_paths)}개 중 {START_IDX}~{END_IDX} 처리 (총 {len(target_paths)}개)")

for chunk_start in range(START_IDX, END_IDX + 1, CHUNK_SIZE):
    chunk_end = min(chunk_start + CHUNK_SIZE - 1, END_IDX)

    out_csv = folder / f"captions_{chunk_start:03d}_{chunk_end:03d}.csv"
    done = load_done_filenames(out_csv)

    print(f"\n=== [{chunk_start}~{chunk_end}] -> {out_csv.name} | 성공 처리(done): {len(done)}개 ===")

    chunk_paths = image_paths[chunk_start-1:chunk_end]

    for global_idx, path in enumerate(tqdm(chunk_paths, desc=f"Captioning {chunk_start}~{chunk_end}"), start=chunk_start):
        if path.name in done:
            continue

        try:
            img = Image.open(path).convert("RGB")
        except Exception as e:
            append_row(out_csv, {"index": global_idx, "filename": path.name, "caption": None, "error": f"image_open_fail: {e}"})
            continue

        caption, err = generate_caption(img)
        append_row(out_csv, {"index": global_idx, "filename": path.name, "caption": caption, "error": err})

        time.sleep(SLEEP_PER_CALL)

    print(f"완료(청크): {chunk_start}~{chunk_end}")